# Scavenger hunt
___

Now it's your turn! Here's the questions I would like you to get the data to answer:

* Which countries use a unit other than ppm to measure any type of pollution? (Hint: to get rows where the value *isn't* something, use "!=")
* Which pollutants have a value of exactly 0?

In order to answer these questions, you can fork this notebook by hitting the blue "Fork Notebook" at the very top of this page (you may have to scroll up).  "Forking" something is making a copy of it that you can edit on your own without changing the original.

In [ ]:
import pandas as pd
from bq_helper import BigQueryHelper as BQH

open_aq = BQH(active_project="bigquery-public-data", 
              dataset_name="openaq")
open_aq.list_tables()



In [ ]:
schema = open_aq.table_schema('global_air_quality')
properties = ('name', 'field_type', 'is_nullable', 'description')
schema_df = pd.DataFrame([[eval('field.' + p) for field in schema] 
              for p in properties]).transpose()
schema_df.columns = properties
schema_df

In [ ]:
#Which countries use a unit other than ppm to measure any type of pollution?

ppm_query = """
    SELECT country AS code, unit
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE unit != 'ppm'
    GROUP BY country, unit
    ORDER BY country ASC"""

ppm_results = open_aq.query_to_pandas_safe(ppm_query)

country_names = pd.read_csv('../input/wikipedia-iso-country-codes.csv')
country_names.columns = ['country', '2code', '3code', 'numcode', 'iso']

ppm_results = pd.merge(ppm_results, country_names, 
                       left_on = 'code', right_on = '2code', 
                       how = 'left')
ppm_results = ppm_results[['code', 'country', 'unit']]
ppm_results

In [ ]:
#Which pollutants have a value of exactly 0?
pollutants_query = """
SELECT pollutant, COUNT(country) AS n_measurements
FROM `bigquery-public-data.openaq.global_air_quality`
WHERE value = 0
GROUP BY pollutant
ORDER BY n_measurements DESC
"""

pollutants_df = open_aq.query_to_pandas_safe(pollutants_query)
pollutants_df

Please feel free to ask any questions you have in this notebook or in the [Q&A forums](https://www.kaggle.com/questions-and-answers)! 

Also, if you want to share or get comments on your kernel, remember you need to make it public first! You can change the visibility of your kernel under the "Settings" tab, on the right half of your screen.